<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Uebungen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung einlesen der Daten 

In diesem Abschnitt müssen Sie nichts machen. Dieser dient nur für das Einlesen und die Bereitstellung der Daten für die folgenden Übungen. 

In [ ]:
# Bibliotheken einmalig installieren
!pip install pyspark pandas pysqlite3 

In [ ]:
#Angabe der Daten für den Pfad
!wget -O create_data.py https://github.com/Fuenfgeld/2022TeamADataEngineeringBC/blob/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Pr%C3%A4sentationen/03-Fortgeschrittene%20Transformationen/create_data.py

--2022-05-09 20:18:50--  https://github.com/Fuenfgeld/2022TeamADataEngineeringBC/blob/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Pr%C3%A4sentationen/03-Fortgeschrittene%20Transformationen/create_data.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘create_data.py’

create_data.py          [ <=>                ] 281.07K  --.-KB/s    in 0.05s   

2022-05-09 20:18:50 (5.85 MB/s) - ‘create_data.py’ saved [287811]



In [ ]:
#Import der nötigen Packete
import sqlite3
import pandas as pd
import pyspark.sql.functions as func
import os

os.system("python3 create_data.py")

256

In [ ]:
## (Py)Spark starten
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Datei Corn einlesen & in Spark-Dataframe schreiben
df_corn = spark.read.option("multiline",True).json('Corn.json')
df_corn.show()


+----+-----+-------+-----------------+----+-------------+
|crop|field|revenue|water_consumption|week|yield_per_sqm|
+----+-----+-------+-----------------+----+-------------+
|corn|    8|      0|               12|   1|            0|
|corn|    9|      0|               10|   2|            0|
|corn|    8|      0|               12|   3|            0|
|corn|    8|      0|               14|   4|            0|
|corn|    8|      0|               14|   5|            5|
|corn|    9|     30|               18|   6|           10|
|corn|    8|      0|               12|   1|            0|
|corn|   10|      0|               10|   2|            0|
|corn|   10|      0|               12|   3|            0|
|corn|    8|     60|               20|   7|           25|
|corn|   10|    150|               26|   8|           25|
|corn|    8|    150|               24|   9|           25|
|corn|    9|    100|               10|  10|           25|
|corn|    8|    150|             null|  11|            0|
+----+-----+--

In [ ]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

(5, 'Barn Ground', 10)
(6, 'Bank', 10)
(7, 'Far Brossler', 20)


Dies ist das erzeugte Dataframe:

    field_id    field_name  area_in_sqm
0         5   Barn Ground           10
1         6          Bank           10
2         7  Far Brossler           20


In [ ]:
#DataFrame für Datenbank erzeugen
df_fields= spark.createDataFrame(df_fields) 
df_fields.show()

+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
+--------+------------+-----------+



Nachdem nun alle Daten geladen wurden können Sie anschließend mit den Übungen beginnen.

#Übungen für Fortgeschrittene 

In dieser Übung müssen Sie Jeremy tatkräftig unter die Arme greifen. 
Nachdem wir bereits Ihm bereits bei dem Anbau des Obst und Gemüse geholen haben, müssen Sie nun für in die Daten für das Getreide auswerten.

Hierfür müssen Sie folgend vorgehen:

1.   Helfen Sie Jeremy dabei sich einen Überblick über die Daten zu verschaffen in dem Sie JOIN verwerden.
2.   




# Transformationen für Fortgeschrittene

Damit Jeremy sich weitere Inforamtionen aus den Daten beschaffen kann,muss er sich erneut einen Überblick verschaffen. 

In [ ]:
df_fields.show(10)
df_corn.show(10)

+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
+--------+------------+-----------+

+----+-----+-------+-----------------+----+-------------+
|crop|field|revenue|water_consumption|week|yield_per_sqm|
+----+-----+-------+-----------------+----+-------------+
|corn|    8|      0|               12|   1|            0|
|corn|    9|      0|               10|   2|            0|
|corn|    8|      0|               12|   3|            0|
|corn|    8|      0|               14|   4|            0|
|corn|    8|      0|               14|   5|            5|
|corn|    9|     30|               18|   6|           10|
|corn|    8|      0|               12|   1|            0|
|corn|   10|      0|               10|   2|            0|
|corn|   10|      0|               12|   3|            0|
|corn|    8|     60|               20|   7|        

Für die kommenden Wochen ist mit erhöhten Niederschlägen zu rechnen. Um zuwissen wann es sich lohnt die Felder zu bewässern, will sich Jeremy einen Überblick über die Durchschnittliche Wassermenge pro Woche verschaffen.

In [ ]:

df_alles_was_angebaut_wird = df_fru_veg.groupBy("week").agg(func.avg("water_consumption"))
df_alles_was_angebaut_wird.show()


NameError: ignored

In [ ]:
#Gesamter Wasserverbrauch pro Pflanze aufsummieren Beispiel GroupBy
gesamt_verbrauch =  df_fru_veg.groupBy("crop").agg(func.sum("water_consumption"))
gesamt_verbrauch.show()

als nächstes Folgen die Beispiel für die Join Bedingung

In [ ]:
#Beispiel JOIN
#Nun will Jeremy wieviele Pflanzen er auf jedem Feld anbauen kann  

gesamt_join = df_fru_veg.join(df_fields, df_fru_veg.field ==  df_fields.field_id,"inner")
gesamt_join.show(30)

In [ ]:

gesamt_join = gesamt_join.withColumn("yield_per_area", col("yield_per_sqm") * col("area_in_sqm"))
gesamt_join.show()


In [ ]:
#Nachdem der Ertrag für die einzelnen Gemüse- oder Obstsorten ermittelt wurde müssen wir noch unseren Netto gewinn für die einzelnen Sachen berechenen.
gesamt_ertrag =  gesamt_join.groupBy("crop").agg(func.sum("yield_per_area"), func.sum("water_consumption"))
gesamt_ertrag = gesamt_ertrag.select(col("crop").alias("Crop"),col("sum(water_consumption)").alias("water_consumption"), col("sum(yield_per_area)").alias("yield_per_area"))
gesamt_ertrag.show()


Hierfür benötogen wir die zuvor ermittelten Liter je Sorte welche wir mit dem Wert von 20Cent verrechnen müssen um die gesanten Bewässerungskosten zu erhalten.

In [ ]:
gesamt_ertrag = gesamt_ertrag.withColumn("netto_yield_per_area",col("yield_per_area") - (col("water_consumption")* 0.2))
df3 = gesamt_ertrag.withColumn("netto_yield_per_area", func.round(gesamt_ertrag["netto_yield_per_area"], 2))
df3.show()

In [ ]:
df3.orderBy(col("netto_yield_per_area").desc()).show(truncate=False)



Nun mach ich noch irgendwas anderes und dann sollte das wenigstens für heute passen 